In [1]:
import Ativo as at
import FileManager as fman
import pickle
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import TradesAnalyser as ta
import seaborn as sns

pd.set_option('display.max_rows', 167) # opcional, questão de visualização para debugar caso 1

In [2]:
fm = fman.FileManager()

In [3]:
with open('AtivoDiaList.pkl', 'rb') as filehandle: # w de write e b de binary
    ativo_dia_list = pickle.load(filehandle) # notar como para salvar precisamos passar a list e o filehandle, mas pra abrir não

In [4]:
an = ta.TradesAnalyser(ativo_dia_list)

In [ ]:
#an.setFilterParameters(gap_threshold=0.1, F_high_threshold=9)
#an.setFilterParameters(prevol_threshold=15000000)
an.runFiltering()

In [ ]:
#an.setSimParameters(start_money = 10000,allocation=0.1)
#an.setAlgoParameters(short_after=0, exit_target=0.4, exit_stop=0.4)
#an.setAlgoParameters(short_after=0.2)
an.runSimulation()
# an.saveTrades('case1_trades.pkl')

#an.openTrades('case1_trades.pkl')

In [ ]:
#an.saveTrades('case4_trades.pkl')

In [ ]:
dfd = an.getFilteredDays() # DataFrame of filtered ativo-dias
dft = an.getTrades() # DataFrame of trades

##### Análise de algumas estratificações

In [ ]:
# CALCULA DATAFRAME DE TRADE RESULTS, QUE VAI PEGAR ELEMENTOS QUE INTERESSAM DA INNER JOIN ENTRE dfd e dft
dftr = pd.merge(dft,dfd,how='inner',on=['name','date'])
dftr = dftr[['name','date','gap','factorF','profit','cum_profit']]
dftr

In [ ]:
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
bins = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2]
gap = np.clip( dftr['gap'], bins[0], bins[-1] )
dftr.groupby(pd.cut(gap, bins)).agg(
    winrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0)/(sum(x>=0) + sum(x<0)) ),
    lossrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0)/(sum(x>=0) + sum(x<0)) ),
    mean=pd.NamedAgg(column='profit',aggfunc=lambda x: x.mean()),
    mean_of_pos=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x>=0].mean()),
    mean_of_neg=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x<0].mean()),
    count=pd.NamedAgg(column='profit',aggfunc='count'),
    positives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0) ),
    negatives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0) )
)

In [ ]:
dftr.groupby(dftr['date'].dt.to_period('M')).agg(
    winrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0)/(sum(x>=0) + sum(x<0)) ),
    lossrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0)/(sum(x>=0) + sum(x<0)) ),
    mean=pd.NamedAgg(column='profit',aggfunc=lambda x: x.mean()),
    mean_of_pos=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x>=0].mean()),
    mean_of_neg=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x<0].mean()),
    count=pd.NamedAgg(column='profit',aggfunc='count'),
    positives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0) ),
    negatives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0) )
)

In [ ]:
an.printSimResults()

In [ ]:
an.plotHistMinsToTrade()

In [ ]:
an.plotEquityCurve(logy=False)

In [ ]:
an.getMaxDrawdown()

In [ ]:
an.getSimResults()

In [ ]:
an.runBootstrap(n_iter=50, replace=False)
an.printBootstrapResults()

In [ ]:
an.getBootstrapResults()

##### Códigos para rodar combinações de parâmetros e otimizar.

In [ ]:
# oficial que gostaríamos de explorar
an.runSimulationGroup(
    prevol_threshold = [800000],
    exit_target=[0.1, 0.2, 0.3, 0.4], 
    exit_stop=[0.1, 0.2, 0.3, 0.4], 
    short_after=[0, 0.1, 0.2, 0.3],
    gap_threshold=[0, 0.1, 0.2],
    F_high_threshold = [1,2,3,4,5]
) # essa combinação daria 960. vamos rodando devagarinho

In [7]:
# an.saveGroupResults('erich01.pkl')
an.openGroupResults('erich01.pkl')

In [ ]:
# rodar para debugar
an.runSimulationGroup(
    prevol_threshold = [15000000],
    exit_target=[0.2], 
    exit_stop=[0.2, 0.3], 
    short_after=[0, 0.1],
    gap_threshold=[0, 0.1],
    F_high_threshold = [1]
)

In [8]:
an.results

,prevol_threshold,open_dolar_threshold,gap_threshold,F_low_threshold,F_high_threshold,short_after,exit_target,exit_stop,start_money,allocation,locate_fee,commission,end_money,profit,max_drawdown,meanmax_drawdown,maxmax_drawdown,minmax_drawdown,n_trades,n_filtered_ativo_days
0,15000000,2,0.0,0,1,0.0,0.2,0.2,10000,0.1,0.02,2,11170.514814,0.117051,0.051159,0.053123,0.104389,0.026856,22,22
1,15000000,2,0.0,0,1,0.0,0.2,0.3,10000,0.1,0.02,2,11681.104810,0.168110,0.044794,0.051653,0.093499,0.032200,22,22
2,15000000,2,0.0,0,1,0.1,0.2,0.2,10000,0.1,0.02,2,11808.764880,0.180876,0.022200,0.027763,0.044581,0.022200,15,22
3,15000000,2,0.0,0,1,0.1,0.2,0.3,10000,0.1,0.02,2,12167.222654,0.216722,0.032177,0.032260,0.032870,0.032177,15,22
4,15000000,2,0.1,0,1,0.0,0.2,0.2,10000,0.1,0.02,2,11170.514814,0.117051,0.051159,0.050020,0.077095,0.022200,22,22
5,15000000,2,0.1,0,1,0.0,0.2,0.3,10000,0.1,0.02,2,11681.104810,0.168110,0.044794,0.050817,0.107058,0.032200,22,22
6,15000000,2,0.1,0,1,0.1,0.2,0.2,10000,0.1,0.02,2,11808.764880,0.180876,0.022200,0.024662,0.043892,0.022200,15,22
7,15000000,2,0.1,0,1,0.1,0.2,0.3,10000,0.1,0.02,2,12167.222654,0.216722,0.032177,0.032329,0.032870,0.032177,15,22
8,15000000,2,0.0,0,1,0.0,0.2,0.2,10000,0.1,0.02,2,11170.514814,0.117051,0.051159,0.053123,0.104389,0.026856,22,22
9,15000000,2,0.0,0,1,0.0,0.2,0.3,10000,0.1,0.02,2,11681.104810,0.168110,0.044794,0.051653,0.093499,0.032200,22,22


##### Descrição simulações já rodadas

##### Testes temporários em geral

In [ ]:
an.getMaxDrawdown()